In [0]:
from pyspark.sql.functions import (
    col, count, avg, min, max, collect_set, explode,
    when, current_timestamp, percent_rank
)
from pyspark.sql import Window

# FACT_RATINGS — optimized for ML training
df_fact_ratings = (
    spark.table("workspace.silver.ratings")
    .select("userId", "movieId", "rating", "timestamp")
)

df_fact_ratings.write.format("delta").mode("overwrite").saveAsTable("workspace.gold.fact_ratings")
print(f"✅ Gold fact_ratings: {spark.table('workspace.gold.fact_ratings').count()} rows")

# DIM_USERS — user behavior profiles
df_dim_users = (
    spark.table("workspace.silver.ratings")
    .groupBy("userId")
    .agg(
        count("rating").alias("total_ratings"),
        avg("rating").alias("avg_rating"),
        min("timestamp").alias("first_rating"),
        max("timestamp").alias("last_rating"),
        count(when(col("rating") >= 4.0, True)).alias("high_ratings_count"),
    )
    .withColumn(
        "rating_percentile",
        percent_rank().over(Window.orderBy("total_ratings"))
    )
    .withColumn(
        "is_power_user",
        when(col("rating_percentile") >= 0.90, True).otherwise(False)  # top 10%
    )
)

df_dim_users.write.format("delta").mode("overwrite").saveAsTable("workspace.gold.dim_users")
print(f"✅ Gold dim_users: {spark.table('workspace.gold.dim_users').count()} rows")

# DIM_MOVIES_ENRICHED — movie details
df_movies = spark.table("workspace.silver.movies_enriched")

df_movie_stats = ( # Add aggregate rating info per movie
    spark.table("workspace.silver.ratings")
    .groupBy("movieId")
    .agg(
        count("rating").alias("rating_count"),
        avg("rating").alias("avg_rating")
    )
)

df_dim_movies = (
    df_movies
    .join(df_movie_stats, on="movieId", how="left")
    .fillna({"rating_count": 0, "avg_rating": 0.0})
)

df_dim_movies.write.format("delta").mode("overwrite").saveAsTable("workspace.gold.dim_movies_enriched")
print(f"✅ Gold dim_movies_enriched: {spark.table('workspace.gold.dim_movies_enriched').count()} rows")


✅ Gold fact_ratings: 32000204 rows


/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


✅ Gold dim_users: 200948 rows
✅ Gold dim_movies_enriched: 32000204 rows


In [0]:
# SAMPLE QUERIES — test the Gold layer
print("\n--- Top 10 Power Users ---")
spark.table("workspace.gold.dim_users").filter("is_power_user = true").orderBy(col("total_ratings").desc()).show(10)

print("\n--- Top 10 Highest Rated Movies (min 50 ratings) ---")
spark.table("workspace.gold.dim_movies_enriched").filter("rating_count >= 50").orderBy(col("avg_rating").desc()).show(10, truncate=False)


--- Top 10 Power Users ---
+------+-------------+------------------+-------------------+-------------------+------------------+------------------+-------------+
|userId|total_ratings|        avg_rating|       first_rating|        last_rating|high_ratings_count| rating_percentile|is_power_user|
+------+-------------+------------------+-------------------+-------------------+------------------+------------------+-------------+
|175325|        33332| 3.077808112324493|2015-12-15 06:59:27|2020-06-12 15:35:00|              4852|               1.0|         true|
| 17035|         9577|2.5678187323796595|2012-12-13 02:21:42|2023-10-12 12:00:17|               934|0.9999950235634272|         true|
| 55653|         9178|3.2802898234909565|2001-08-05 18:10:38|2015-12-24 20:44:49|              2374|0.9999900471268544|         true|
|123465|         9044| 2.528858911985847|2010-11-28 02:03:36|2023-10-12 17:01:35|              1834|0.9999850706902815|         true|
|171795|         9016|3.1819543034